## LangChain Chains

In [9]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, OpenAI, LLMChain

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [13]:
template = """
Extract name of a person and language of message from the input.

Format the output as JSON with the following keys:
name
language

text: {input}
"""

llm = OpenAI(temperature=0)
prompt_template = PromptTemplate.from_template(template=template)
name_lang_chain = LLMChain(llm=llm, prompt=prompt_template)
name_lang_chain.predict(input="Herr Josef Braun ist am 22.09.1999 geboren.")

'\n{\n    "name": "Josef Braun",\n    "language": "German"\n}'

In [14]:
response_template = """
You are an AI assistant generating greeting message for the beginning of an e-mail. 
Propose greeting using provided name and language.

text: {input}
"""
greeting_template = PromptTemplate(input_variables=["input"], template=response_template)
greeting_chain = LLMChain(llm=llm, prompt=greeting_template)
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[name_lang_chain, review_chain], verbose=True)

overall_chain.run(input="Herr Josef Braun ist am 22.09.1999 geboren.")



> Entering new SimpleSequentialChain chain...

{
    "name": "Josef Braun",
    "language": "German"
}

Guten Tag Josef Braun,

Ich hoffe, es geht Ihnen gut. Ich wollte Ihnen nur eine kurze E-Mail schreiben, um mich vorzustellen und Ihnen mitzuteilen, dass ich Ihr neuer AI-Assistent bin. Ich freue mich darauf, Ihnen bei all Ihren Aufgaben und Anfragen behilflich zu sein. Zögern Sie nicht, mich jederzeit zu kontaktieren.

Mit freundlichen Grüßen,
[Your Name]

> Finished chain.


'\nGuten Tag Josef Braun,\n\nIch hoffe, es geht Ihnen gut. Ich wollte Ihnen nur eine kurze E-Mail schreiben, um mich vorzustellen und Ihnen mitzuteilen, dass ich Ihr neuer AI-Assistent bin. Ich freue mich darauf, Ihnen bei all Ihren Aufgaben und Anfragen behilflich zu sein. Zögern Sie nicht, mich jederzeit zu kontaktieren.\n\nMit freundlichen Grüßen,\n[Your Name]'